In [1]:
from elasticsearch import Elasticsearch
import pandas as pd

# Connexion à Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200,'scheme': 'http'}])

In [2]:
resp = es.search(index="notes", query={"match_all": {}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits'][0:7]:
    print(hit['_source'])

Got 10000 Hits:
{'Text': 'i do not feel miserable at all because my family is not the type that celebrates eid', 'Emotion': 'sadness', 'Tokens': "['miserable', 'family', 'type', 'celebrates', 'eid']", 'patient_firstname': 'Young', 'patient_lastname': 'Dawn', 'emotion': 'sadness', 'confidence': 0.9320817911161929, 'date': '2023-02-22', 'patient_left': False}
{'Text': 'i might go get a car wash if i am feeling really generous my car needs it', 'Emotion': 'happy', 'Tokens': "['might', 'car', 'wash', 'generous', 'car']", 'patient_firstname': 'Hall', 'patient_lastname': 'Blake', 'emotion': 'happy', 'confidence': 0.7843972261874939, 'date': '2022-10-02', 'patient_left': True}
{'Text': 'i still feel sleep deprived she is almost sleeping through the night giving us', 'Emotion': 'sadness', 'Tokens': "['sleep', 'deprived', 'almost', 'sleeping', 'night', 'giving', 'u']", 'patient_firstname': 'Turner', 'patient_lastname': 'Anthony', 'emotion': 'sadness', 'confidence': 0.9554573722591212, 'date': '

/tmp/ipykernel_400147/3224154290.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resp = es.search(index="notes", query={"match_all": {}})


1 - Répartition des sentiments par patient

In [7]:
def get_sentiment_distribution(es,patient_firstname, patient_lastname):
    query = {
            "size": 10,
            "query": {
                "bool": {
                    "must": [
                        {"term": {"patient_firstname": patient_firstname}},
                        {"term": {"patient_lastname": patient_lastname}}
                    ]
                }
            },
            "aggs": {
                "sentiment_distribution": {
                    "terms": {"field": "Emotion"}
                }
            }
        }
    

    response = es.search(index="notes", body=query)

    buckets = response["aggregations"]["sentiment_distribution"]["buckets"]

    df = pd.DataFrame(buckets, columns=["key", "doc_count"])
    return df

print(get_sentiment_distribution(es,'Samantha','Smith'))

/tmp/ipykernel_400147/1587608412.py:20: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="notes", body=query)


BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'Text fields are not optimised for operations that require per-document field data like aggregations and sorting, so these operations are disabled by default. Please use a keyword field instead. Alternatively, set fielddata=true on [Emotion] in order to load field data by uninverting the inverted index. Note that this can use significant memory.')